# Director-AI — Protect any LLM in 10 lines

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/anulum/director-ai/blob/main/notebooks/quickstart.ipynb)
[![PyPI](https://img.shields.io/pypi/v/director-ai.svg)](https://pypi.org/project/director-ai/)

**Director-AI** is a real-time LLM hallucination guardrail. It scores every LLM output
for coherence against your own knowledge base — and can halt generation mid-stream if
coherence drops below threshold.

This notebook covers:
1. Install + basic scoring
2. Guard an LLM (no API key needed)
3. Streaming halt demo
4. Domain presets
5. Full agent pipeline

In [ ]:
!pip install -q director-ai

## 1. Score a Response

The `CoherenceScorer` computes a dual-entropy score: logical divergence (contradiction)
and factual divergence (deviation from your facts). Both must pass.

In [ ]:
from director_ai.core import CoherenceScorer, GroundTruthStore

store = GroundTruthStore()
store.add("capital", "Paris is the capital of France.")
store.add("sky color", "The sky is blue due to Rayleigh scattering.")

scorer = CoherenceScorer(threshold=0.6, ground_truth_store=store, use_nli=False)

tests = [
    ("What is the capital of France?", "The capital of France is Paris."),
    ("What is the capital of France?", "The capital of France is Berlin."),
    ("What color is the sky?", "The sky is blue."),
    ("What color is the sky?", "The sky is green, obviously."),
]

for prompt, llm_output in tests:
    approved, score = scorer.review(prompt, llm_output)
    status = "PASS" if approved else "BLOCKED"
    print(f"[{status}] {llm_output}")
    print(f"  coherence={score.score:.3f}  h_logical={score.h_logical:.2f}  h_factual={score.h_factual:.2f}\n")

## 2. Guard an LLM

The `guard()` wrapper intercepts any OpenAI-compatible client. Here we use `MockGenerator` so no API key is needed.

In [ ]:
from director_ai import CoherenceAgent

agent = CoherenceAgent()  # MockGenerator — no API key required
result = agent.process("What color is the sky?")

print(f"Output:  {result.output}")
print(f"Halted:  {result.halted}")
if result.coherence:
    print(f"Score:   {result.coherence.score:.3f}")
    print(f"Approved: {result.coherence.approved}")

## 3. Streaming Halt

The `StreamingKernel` monitors coherence **token-by-token** and halts generation
the moment it degrades. Three halt mechanisms:
- **Hard limit** — single token below absolute floor
- **Sliding window** — rolling average drops
- **Downward trend** — coherence decay over N tokens

In [ ]:
from director_ai.core import StreamingKernel

kernel = StreamingKernel(
    hard_limit=0.35,
    window_size=5,
    window_threshold=0.45,
)

# Synthetic tokens: starts accurate, drifts into hallucination
tokens = [
    "Water", " boils", " at", " 100", " C.",
    " But", " actually", " the", " real",
    " temperature", " is", " negative", " forty", ".",
]
scores = [
    0.92, 0.90, 0.91, 0.89, 0.88,
    0.72, 0.55, 0.40, 0.30, 0.15, 0.10, 0.08, 0.05, 0.03,
]

idx = 0
def coherence_cb(tok):
    global idx
    s = scores[min(idx, len(scores) - 1)]
    idx += 1
    return s

session = kernel.stream_tokens(iter(tokens), coherence_cb)

for ev in session.events:
    marker = " <<<HALT" if ev.halted else ""
    print(f"  [{ev.index:2d}] {ev.coherence:.3f}  {ev.token!r}{marker}")

print(f"\nHalted: {session.halted}")
if session.halted:
    print(f"Reason: {session.halt_reason}")
print(f"Tokens: {session.token_count}/{len(tokens)}")

## 4. Domain Presets

Director-AI ships 8 domain profiles with tuned thresholds:

In [ ]:
from director_ai.core.config import DirectorConfig

profiles = ["fast", "thorough", "research", "medical", "finance", "legal", "creative", "customer_support"]

print(f"{'Profile':<18} {'Threshold':>9} {'Hard Limit':>10} {'NLI':>5} {'w_logic':>7} {'w_fact':>7}")
print("-" * 60)
for name in profiles:
    cfg = DirectorConfig.from_profile(name)
    print(
        f"{name:<18} {cfg.coherence_threshold:>9.2f} {cfg.hard_limit:>10.2f}"
        f" {str(cfg.use_nli):>5} {cfg.w_logic:>7.1f} {cfg.w_fact:>7.1f}"
    )

## Next Steps

- [Full docs](https://anulum.github.io/director-ai/) — scoring, streaming, deployment
- [GitHub](https://github.com/anulum/director-ai) — source, issues, contributing
- [PyPI](https://pypi.org/project/director-ai/) — `pip install director-ai[nli]` for NLI scoring
- [HF Spaces demo](https://huggingface.co/spaces/anulum/director-ai-guardrail) — try it in your browser

In [ ]:
!director-ai version